In [1]:
#toggle relevant models
# with open('../models_pkl/random_forest.pkl', 'rb') as fit_rf:
#     clf = pickle.load(fit_rf)

In [3]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from tensorflow.keras import datasets, layers, models, regularizers, initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.io import imshow
from skimage.transform import rotate
from skimage.filters.edges import convolve

from skimage.io import imshow, imread
from skimage.transform import resize
from skimage.color import rgb2gray

import dill as pickle

np.random.seed(33)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [24]:
df_val = pd.read_csv('../data/extra_labels.csv')

In [25]:
def pick_ylabels(column):
    y_val = df_val[column].copy().astype('category').cat.codes
    return (y_val.values)

In [26]:
loaded2 = np.load('../data/bw_images.npz')
X_val = loaded2['c']

In [27]:
val_true = pick_ylabels('masterCategory')[:20]

In [28]:
X_val = X_val.reshape(-1,80,60,1)[:20]

In [29]:
# get the predictions for the test data
predicted_classes = model.predict_classes(X_val)

# get the indices to be plotted
correct = np.nonzero(predicted_classes==val_true)[0]
incorrect = np.nonzero(predicted_classes!=val_true)[0]

In [30]:
df_val.iloc[incorrect, :]

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
9,55315,Men,Apparel,Socks,Socks,White,Summer,2016.0,Casual,Smartoe Men Pack of 3 Socks
18,43973,Women,Personal Care,Fragrance,Perfume and Body Mist,White,Spring,2017.0,Casual,DKNY Women Fresh Blossom 50 ml Perfume


### Test on Non-Sample

In [5]:
from tensorflow.keras.models import Sequential, load_model
model = load_model('../json/cnn-main.h5')

In [14]:
# from PIL import Image
# from skimage import transform,io, color
# from os import listdir
# import numpy as np
def process_images(directory):
    from PIL import Image
    from skimage import transform,io, color
    from os import listdir
    import numpy as np
    upload_array = []
    pic_list = [f for f in listdir(directory) if f.endswith('.png')]

    for pic in pic_list:
        fn = r'{}/{}'.format(directory, pic)
        img = io.imread(fn)
        img = transform.rescale(img, 3.0 / 4.0, anti_aliasing=False)
        if img.shape[2]==4:
            img = color.rgba2rgb(img)
            
        img_new = transform.resize(img, (80,60,3))
        upload_array.append(img_new.ravel())

    all_uploads = np.stack(upload_array)
    bw_uploads = all_uploads.reshape(-1, 80, 60, 3).mean(3).reshape(-1, 80*60).astype('float16') 
#     all_uploads = all_uploads.reshape(-1,80,60,3)
#     bw_uploads = bw_uploads.reshape(-1,80,60,1)
    return (all_uploads, bw_uploads)

In [15]:
def predict_master_cat(upload_array):
    predicted_category = model.predict_classes(upload_array)
    probability = np.round(model.predict_proba(upload_array),3)
    arr = np.zeros([len(predicted_category), 2])
    for i in range(len(predicted_category)):
        arr[i,:] = [predicted_category[i], probability[i][predicted_category[i]]]
    return arr

In [16]:
__, bw_new = process_images('image_uploads/')

In [17]:
bw_new = bw_new.reshape(-1,80,60,1)

In [18]:
new_predictions = np.round(predict_master_cat(bw_new),3)

In [19]:
new_predictions

array([[0.   , 0.765],
       [0.   , 0.737],
       [0.   , 0.744],
       [0.   , 0.718],
       [0.   , 0.856],
       [0.   , 0.743],
       [0.   , 0.728],
       [0.   , 0.815],
       [0.   , 0.824]])

In [20]:
model.predict_proba(bw_new)

array([[7.6521778e-01, 2.3324341e-01, 2.2964305e-05, 1.5158394e-03],
       [7.3654658e-01, 2.6217195e-01, 1.5020367e-05, 1.2664918e-03],
       [7.4411660e-01, 2.5441214e-01, 2.1152398e-05, 1.4502029e-03],
       [7.1814048e-01, 2.8026146e-01, 2.8245644e-05, 1.5697915e-03],
       [8.5559654e-01, 1.4295562e-01, 1.7161905e-05, 1.4306670e-03],
       [7.4342293e-01, 2.5529781e-01, 1.4275909e-05, 1.2649909e-03],
       [7.2840315e-01, 2.6993889e-01, 2.0614008e-05, 1.6374001e-03],
       [8.1490481e-01, 1.8122782e-01, 2.6256208e-05, 3.8412041e-03],
       [8.2375598e-01, 1.7400430e-01, 5.0187627e-05, 2.1895212e-03]],
      dtype=float32)